In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # visualizations
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
# creating fonction that normalize the sub-categs
def norm(tab): 
    if len(tab) > 1:
        max  = np.array(tab).max()
        min = np.array(tab).min()
        return [round((x - min) / (max-min) ,3) for x in tab]
    else : 
        return [1]

def norm_categ_lvl2(dict_categ): 
    for k,v in dict_categ.items(): 
        tab = norm([v for k,v  in v.items()])
        dict_categ[k] ={k:v for (k,_),v in zip(v.items(),tab)}
    return dict_categ

In [ ]:
data_csv = "/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Oct.csv"
raw_data = pd.read_csv(data_csv, nrows = 5000000)
raw_data.head(10)

In [ ]:
raw_data.dropna(inplace=True)
raw_data["category_lvl1"] = raw_data["category_code"].apply(lambda x : x.split(".")[0])
raw_data["category_lvl2"] = raw_data["category_code"].apply(lambda x : x.split(".")[1])
raw_data.reset_index(drop = True,inplace=True)
#Applying data analysis only on purshased products
only_purchases = raw_data.loc[raw_data.event_type == 'purchase']

In [ ]:
purchases_with_brands = only_purchases.loc[only_purchases.brand.notnull()]
top_sellers_count = purchases_with_brands.groupby('brand').brand.agg([len]).sort_values(by="len", ascending=False)
top_sellers_count.reset_index(inplace=True)
top_sellers_count.rename(columns={"len" : "n°sales/brand"}, inplace=True)

In [ ]:
top_sellers_count.describe()

In [ ]:
top_sellers_count

Visualization

In [ ]:
# Visualization

plt.figure(figsize=(10,10))
plt.pie(top_sellers_count["n°sales/brand"].head(10),labels=top_sellers_count["brand"].head(10))
plt.axis('equal')